In [15]:
import requests

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection

model_id = "IDEA-Research/grounding-dino-base"

processor = AutoProcessor.from_pretrained(model_id)
device = "cuda"
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)


preprocessor_config.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/933M [00:00<?, ?B/s]

Detected 1 with confidence 0.44 at location [773.7, 340.5, 818.9, 384.7]


In [40]:
from pathlib import Path

file_list = sorted(Path("plot/f1").glob("*jpg"))
image_url = str(file_list[3])
image = Image.open(image_url)
# Check for cats and remote controls
text = "a pen."

inputs = processor(images=image, text=text, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API

target_sizes = torch.tensor([image.size[::-1]])

results = processor.image_processor.post_process_object_detection(

    outputs, threshold=0.35, target_sizes=target_sizes

)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):

    box = [round(i, 1) for i in box.tolist()]
    conf = round(score.item(), 2)

    print(f"Detected {label.item()} with confidence " f"{round(score.item(), 2)} at location {box}")

Detected 1 with confidence 0.7 at location [753.3, 467.8, 921.2, 552.5]


In [41]:
import ultralytics
from ultralytics.utils.plotting import Annotator

annotator = Annotator(image)
annotator.box_label(box, label=str(conf), color=(0, 204, 0)) # The bounding box coordinates (x1, y1, x2, y2)

import cv2
result = annotator.result()
cv2.imwrite("plot.jpg", cv2.cvtColor(result, cv2.COLOR_RGB2BGR), [cv2.IMWRITE_JPEG_QUALITY, 100])

True